In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import torch
from torch import nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import init

Here we put the data into a data frame

In [2]:
trainDF = pd.read_csv('train.csv')

In [3]:
labels = trainDF['label']
x = trainDF.drop('label', axis = 1)

In [4]:
x.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create an SVM model using the rbf kernel to identify digits

In [5]:
model = SVC()

In [6]:
model.fit(x, labels)

SVC()

In [7]:
testDF = pd.read_csv('test.csv')
testDF.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
preds = model.predict(testDF)

In [9]:
print(preds.shape)

(28000,)


In [10]:
l = [i for i in range(1,28001)]

In [11]:
mydict = {'Imageid': l, 'Label' : preds}

In [12]:
results = pd.DataFrame(mydict, index = mydict['Imageid'])

In [13]:
results.to_csv('submission.csv', index = False)

Begin CNN solution

In [44]:
class MnistDataSet(Dataset):
    def __init__(self, df):
        self.mnistDF = df
        self.labels = self.mnistDF.label
        
    def __len__(self):
        return len(self.mnistDF)
    
    def __getitem__(self, index):
        grid = np.array(self.mnistDF.iloc[index, 1:])
        label = self.labels[index]
        grid = grid.reshape((28,28))
        grid = torch.tensor(grid)
        grid = grid.unsqueeze(0)
        return grid, label
        

In [45]:
class DigitNet(nn.Module):
    def __init__(self):
        super().__init__()
        conv_layers = []
        
        #first layer
        self.conv1 = nn.Conv2d(1,8, kernel_size = 2, stride = 1, padding = 1)
        self.relu1 = nn.ReLU()
        self.normalize1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a = 0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.normalize1]
        
        #second layer
        self.conv2 = nn.Conv2d(8,16, kernel_size = 2, stride = 1, padding = 0)
        self.relu2 = nn.ReLU()
        self.normalize2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a = 0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.normalize2]
        
        #third layer
        self.conv3 = nn.Conv2d(16,32, kernel_size = 2, stride = 1, padding = 0)
        self.relu3 = nn.ReLU()
        self.normalize3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a = 0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.normalize3]
        
        self.ap = nn.AdaptiveAvgPool2d(output_size = 1)
        self.lin = nn.Linear(in_features = 32, out_features = 10)
        
        self.conv = nn.Sequential(*conv_layers)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        return x

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = DigitNet()

In [47]:
myDataObject = MnistDataSet(trainDF)
trainDataLoader = torch.utils.data.DataLoader(myDataObject, batch_size = 32, shuffle = True)

In [48]:
def training(model, trainDL, num_epochs):
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = .01)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for data in trainDL:
            feats, labels = data[0].to(device), data[1].to(device)
            
            mean, stddev = feats.float().mean(), feats.float().std()
            
            feats = (feats-mean)/stddev
            
            optimizer.zero_grad()
            
            outputs = model(feats)
            labels = labels.long()
            loss = loss_func(outputs, labels)
            optimizer.zero_grad
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            _, pred = torch.max(outputs, 1)
            correct += (pred == labels).sum().item()
            total += feats.shape[0]
        
        avg_loss = running_loss/len(trainDL)
        accuracy = correct/total
        
        print(f'Epoch: {epoch+1} Loss: {avg_loss:.2f}, Accuracy: {accuracy:.2f}')
    print('Finished!')
training(model, trainDataLoader, 5)

Epoch: 0 Loss: 0.63, Accuracy: 0.80
Epoch: 1 Loss: 0.27, Accuracy: 0.92
Epoch: 2 Loss: 0.23, Accuracy: 0.93
Epoch: 3 Loss: 0.21, Accuracy: 0.93
Epoch: 4 Loss: 0.20, Accuracy: 0.94
Finished!
